# Лабораторная работа 2
Выполнили: Елизарова Юлия, Игошин Андрей, Кипаренко Илья, Кириленко Юлия, Клыков Вячеслав, Лепигина Анастасия

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import MinMaxScaler

In [2]:
import sparkmonitor
##!jupyter nbextension install sparkmonitor --py --user --symlink 
#!jupyter nbextension enable sparkmonitor --py --user            
#!jupyter serverextension enable --py --user sparkmonitor
#!ipython profile create && echo "c.InteractiveShellApp.extensions.append('sparkmonitor.kernelextension')" >>  $(ipython profile locate default)/ipython_kernel_config.py

In [3]:
conf.setAppName('ExtensionTestingApp')
conf.setMaster('local[*]')
from pyspark import SparkContext
sc=SparkContext.getOrCreate(conf=conf) #Start the spark context

In [4]:
spark = SparkSession.builder.appName('MovieRatingsProject').getOrCreate()

In [5]:
info = spark.read.csv('ml-100k/u.info',inferSchema=True,sep='\t')
info.show()

+--------------+
|           _c0|
+--------------+
|     943 users|
|    1682 items|
|100000 ratings|
+--------------+



In [6]:
from pyspark.sql.functions import from_unixtime, year

Датасет содержит 100 000 оценок 934 пользователями 1682 фильмов. Каждый пользователь оценил не менее 20 фильмов.

In [7]:
ratings = spark.read.csv('ml-100k/u.data',inferSchema=True,sep='\t')
ratings = ratings.withColumnRenamed('_c0','user_id').withColumnRenamed('_c1','movie_id').withColumnRenamed('_c2','rating').withColumnRenamed('_c3','timestamp')
# ratings.show()

In [8]:
ratings = ratings.withColumn('year_rated', year(from_unixtime('timestamp')))

In [9]:
ratings.show()

+-------+--------+------+---------+----------+
|user_id|movie_id|rating|timestamp|year_rated|
+-------+--------+------+---------+----------+
|    196|     242|     3|881250949|      1997|
|    186|     302|     3|891717742|      1998|
|     22|     377|     1|878887116|      1997|
|    244|      51|     2|880606923|      1997|
|    166|     346|     1|886397596|      1998|
|    298|     474|     4|884182806|      1998|
|    115|     265|     2|881171488|      1997|
|    253|     465|     5|891628467|      1998|
|    305|     451|     3|886324817|      1998|
|      6|      86|     3|883603013|      1998|
|     62|     257|     2|879372434|      1997|
|    286|    1014|     5|879781125|      1997|
|    200|     222|     5|876042340|      1997|
|    210|      40|     3|891035994|      1998|
|    224|      29|     3|888104457|      1998|
|    303|     785|     3|879485318|      1997|
|    122|     387|     5|879270459|      1997|
|    194|     274|     2|879539794|      1997|
|    291|    

Информация о каждом фильме включает в себя название, даты выхода в прокат и появления в магазинах, ссылку на IMDB и жанр. Причем один фильм может быть отнесен к нескольким жанрам. 

In [10]:
movies = spark.read.csv('ml-100k/u.item', inferSchema=True, sep='|')
names = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
for i in range(len(names)):
    movies = movies.withColumnRenamed('_c'+str(i), names[i])


In [11]:
from pyspark.sql.functions import to_date

In [12]:
movies = movies.withColumn('year_released', year(to_date('release_date', 'dd-MMM-yyyy')))

In [13]:
movies.show()

+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+-------------+
|movie_id|         movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|year_released|
+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+-------------+
|       1|    Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|         1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|   

Информация о каждом пользователе включает в себя возраст, пол, профессию и почтовый индекс.

In [14]:
users = spark.read.csv('ml-100k/u.user', inferSchema=True, sep='|')
names = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
for i in range(len(names)):
    users = users.withColumnRenamed('_c'+str(i), names[i])
users.show()

+-------+---+------+-------------+--------+
|user_id|age|gender|   occupation|zip_code|
+-------+---+------+-------------+--------+
|      1| 24|     M|   technician|   85711|
|      2| 53|     F|        other|   94043|
|      3| 23|     M|       writer|   32067|
|      4| 24|     M|   technician|   43537|
|      5| 33|     F|        other|   15213|
|      6| 42|     M|    executive|   98101|
|      7| 57|     M|administrator|   91344|
|      8| 36|     M|administrator|   05201|
|      9| 29|     M|      student|   01002|
|     10| 53|     M|       lawyer|   90703|
|     11| 39|     F|        other|   30329|
|     12| 28|     F|        other|   06405|
|     13| 47|     M|     educator|   29206|
|     14| 45|     M|    scientist|   55106|
|     15| 49|     F|     educator|   97301|
|     16| 21|     M|entertainment|   10309|
|     17| 30|     M|   programmer|   06355|
|     18| 35|     F|        other|   37212|
|     19| 40|     M|    librarian|   02138|
|     20| 42|     F|    homemake

In [15]:
stringIndexer = StringIndexer(inputCol="gender", outputCol="indexed_gender", handleInvalid='error')
model = stringIndexer.fit(users)
users = model.transform(users)
encoder = OneHotEncoder(inputCol="indexed_gender", outputCol='gender_feature')
users = encoder.transform(users)

stringIndexer = StringIndexer(inputCol="occupation", outputCol="indexed_occupation", handleInvalid='error')
model = stringIndexer.fit(users)
users = model.transform(users)
encoder = OneHotEncoder(inputCol="indexed_occupation", outputCol='occupation_feature')
users = encoder.transform(users)

In [16]:
users.show()

+-------+---+------+-------------+--------+--------------+--------------+------------------+------------------+
|user_id|age|gender|   occupation|zip_code|indexed_gender|gender_feature|indexed_occupation|occupation_feature|
+-------+---+------+-------------+--------+--------------+--------------+------------------+------------------+
|      1| 24|     M|   technician|   85711|           0.0| (1,[0],[1.0])|              11.0|   (20,[11],[1.0])|
|      2| 53|     F|        other|   94043|           1.0|     (1,[],[])|               1.0|    (20,[1],[1.0])|
|      3| 23|     M|       writer|   32067|           0.0| (1,[0],[1.0])|               7.0|    (20,[7],[1.0])|
|      4| 24|     M|   technician|   43537|           0.0| (1,[0],[1.0])|              11.0|   (20,[11],[1.0])|
|      5| 33|     F|        other|   15213|           1.0|     (1,[],[])|               1.0|    (20,[1],[1.0])|
|      6| 42|     M|    executive|   98101|           0.0| (1,[0],[1.0])|               8.0|    (20,[8],

In [17]:
genres = ['unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

Для получения целостного датасета было принято решение собрать датафреймы с оценками и информацией о фильмах и пользователях в один.

In [18]:
joined_df = ratings.join(users, ["user_id"], 'outer').drop('zip_code', 'timestamp')
joined_df = joined_df.join(movies, ["movie_id"], 'outer').drop('movie_title', 'release_date', 'video_release_date', 'IMDb_URL')

In [19]:
from pyspark.sql.functions import when

In [20]:
joined_df = joined_df.withColumn('rated_released_year_diff', when(joined_df.year_rated - joined_df.year_released > 0, joined_df.year_rated - joined_df.year_released ).otherwise(0))

In [22]:
joined_df.show(n=10)

+--------+-------+------+----------+---+------+----------+--------------+--------------+------------------+------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+-------------+------------------------+
|movie_id|user_id|rating|year_rated|age|gender|occupation|indexed_gender|gender_feature|indexed_occupation|occupation_feature|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|year_released|rated_released_year_diff|
+--------+-------+------+----------+---+------+----------+--------------+--------------+------------------+------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+-------------+------------------------+
|     148|    251|     2|      1998| 28|

Категориальные значения (профессия, пол) были оформлены при помощи one-hot encoding.

In [23]:


# mmScaler = MinMaxScaler(inputCol="age", outputCol="scaled_age")
# model = mmScaler.fit(joined_df)
# model.transform(joined_df)

# joined_df = joined_df.drop("user_id", "movie_id", "gender", "indexed_gender", "occupation", "indexed_occupation", "age")
# joined_df.show()
joined_df.show()

KeyboardInterrupt: 

In [24]:
avg_ratings_df = joined_df.select(['user_id', 'rating']  + genres)

for genre in genres:
    avg_ratings_df = avg_ratings_df.withColumn('rating {}'.format(genre), avg_ratings_df[genre] * avg_ratings_df.rating)

genre_ratings = list(map(lambda genre: 'rating {}'.format(genre), genres))
df1 = avg_ratings_df.groupBy('user_id').sum(*genres)
df2 = avg_ratings_df.groupBy('user_id').sum(*genre_ratings)

avg_ratings_df = df1.join(df2, ['user_id'])
for genre in genres:
    sum_rating_col = 'sum(rating {})'.format(genre)
    count_col = 'sum({})'.format(genre)
    avg_ratings_df = avg_ratings_df.withColumn('avg({})'
                                   .format(genre), avg_ratings_df[sum_rating_col] / avg_ratings_df[count_col])
    avg_ratings_df = avg_ratings_df.drop(sum_rating_col, count_col)

avg_ratings_df = avg_ratings_df.fillna(value=0)
avg_ratings_df.show()
avg_ratings_df.toPandas().to_csv('~/avg_ratings.csv')

+-------+------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------+
|user_id|avg(unknown)|       avg(Action)|    avg(Adventure)|    avg(Animation)|   avg(Children's)|       avg(Comedy)|        avg(Crime)|  avg(Documentary)|        avg(Drama)|avg(Fantasy)|   avg(Film-Noir)|       avg(Horror)|      avg(Musical)|      avg(Mystery)|      avg(Romance)|       avg(Sci-Fi)|     avg(Thriller)|          avg(War)|avg(Western)|
+-------+------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------+-----------------+------------------+------------------+------------------+------------------+---------------

In [25]:
average_movie_rating = ratings.select('movie_id', 'rating')
average_movie_rating = average_movie_rating.groupBy('movie_id').avg('rating')

average_movie_rating.toPandas().to_csv('~/average_movie_rating.csv')
average_movie_rating.show()

+--------+------------------+
|movie_id|       avg(rating)|
+--------+------------------+
|     496| 4.121212121212121|
|     471|3.6108597285067874|
|     463| 3.859154929577465|
|     148|          3.203125|
|    1342|               2.5|
|     833| 3.204081632653061|
|    1088| 2.230769230769231|
|    1591|3.1666666666666665|
|    1238|             3.125|
|    1580|               1.0|
|    1645|               4.0|
|     392|3.5441176470588234|
|     623| 2.923076923076923|
|     540| 2.511627906976744|
|     858|               1.0|
|     737| 2.983050847457627|
|     243|2.4393939393939394|
|    1025|2.9318181818181817|
|    1084| 3.857142857142857|
|    1127| 2.909090909090909|
+--------+------------------+
only showing top 20 rows



In [26]:
popularity = joined_df.select('movie_id')
popularity = popularity.groupBy('movie_id').count()
popularity = popularity.withColumnRenamed('count', 'popularity').fillna(0)

popularity.toPandas().to_csv('~/movie_popularity.csv')
popularity.show()

+--------+----------+
|movie_id|popularity|
+--------+----------+
|     148|       128|
|     463|        71|
|     471|       221|
|     496|       231|
|     833|        49|
|    1088|        13|
|    1238|         8|
|    1342|         2|
|    1580|         1|
|    1591|         6|
|    1645|         1|
|     243|       132|
|     392|        68|
|     540|        43|
|     623|        39|
|     737|        59|
|     858|         3|
|     897|         2|
|    1025|        44|
|    1084|        21|
+--------+----------+
only showing top 20 rows



In [29]:
joined_df = joined_df.join(average_movie_rating, 'movie_id')

In [30]:
joined_df = joined_df.join(popularity, ['movie_id'])

In [31]:
joined_df = joined_df.join(avg_ratings_df, ['user_id'])

In [32]:

final_df = joined_df.drop("gender", "indexed_gender", "occupation", "indexed_occupation", "age")

In [39]:
#final_df = final_df.drop('avg(rating)', 'popularity')
#final_df= final_df.join(popularity, 'movie_id')
final_df= final_df.join(average_movie_rating, 'movie_id')
final_df.columns

['movie_id',
 'user_id',
 'rating',
 'year_rated',
 'gender_feature',
 'occupation_feature',
 'unknown',
 'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western',
 'year_released',
 'rated_released_year_diff',
 'avg(unknown)',
 'avg(Action)',
 'avg(Adventure)',
 'avg(Animation)',
 "avg(Children's)",
 'avg(Comedy)',
 'avg(Crime)',
 'avg(Documentary)',
 'avg(Drama)',
 'avg(Fantasy)',
 'avg(Film-Noir)',
 'avg(Horror)',
 'avg(Musical)',
 'avg(Mystery)',
 'avg(Romance)',
 'avg(Sci-Fi)',
 'avg(Thriller)',
 'avg(War)',
 'avg(Western)',
 'popularity',
 'avg(rating)']

In [38]:
print(popularity.columns)

['movie_id', 'popularity']


In [41]:
joined_df.unpersist()

DataFrame[user_id: int, movie_id: int, rating: int, year_rated: int, age: int, gender: string, occupation: string, indexed_gender: double, gender_feature: vector, indexed_occupation: double, occupation_feature: vector, unknown: int, Action: int, Adventure: int, Animation: int, Children's: int, Comedy: int, Crime: int, Documentary: int, Drama: int, Fantasy: int, Film-Noir: int, Horror: int, Musical: int, Mystery: int, Romance: int, Sci-Fi: int, Thriller: int, War: int, Western: int, year_released: int, rated_released_year_diff: int, avg(rating): double, popularity: bigint, avg(rating): double, popularity: bigint, avg(unknown): double, avg(Action): double, avg(Adventure): double, avg(Animation): double, avg(Children's): double, avg(Comedy): double, avg(Crime): double, avg(Documentary): double, avg(Drama): double, avg(Fantasy): double, avg(Film-Noir): double, avg(Horror): double, avg(Musical): double, avg(Mystery): double, avg(Romance): double, avg(Sci-Fi): double, avg(Thriller): double, 

In [42]:
info.unpersist()
users.unpersist()
movies.unpersist()
average_movie_rating.unpersist()
avg_ratings_df.unpersist()
popularity.unpersist()

DataFrame[movie_id: int, popularity: bigint]

In [44]:
final_df.columns

['movie_id',
 'user_id',
 'rating',
 'year_rated',
 'gender_feature',
 'occupation_feature',
 'unknown',
 'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western',
 'year_released',
 'rated_released_year_diff',
 'avg(unknown)',
 'avg(Action)',
 'avg(Adventure)',
 'avg(Animation)',
 "avg(Children's)",
 'avg(Comedy)',
 'avg(Crime)',
 'avg(Documentary)',
 'avg(Drama)',
 'avg(Fantasy)',
 'avg(Film-Noir)',
 'avg(Horror)',
 'avg(Musical)',
 'avg(Mystery)',
 'avg(Romance)',
 'avg(Sci-Fi)',
 'avg(Thriller)',
 'avg(War)',
 'avg(Western)',
 'popularity',
 'avg(rating)']

In [43]:
final_df.count()

KeyboardInterrupt: 

In [40]:
print(final_df.count())
final_df = final_df.dropna()
print(final_df.count())

100000
99991


In [46]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import NaiveBayes

In [47]:
answer_col = 'rating'
features_col = 'features'
input_cols = [col for col in final_df.columns if col not in [answer_col, "user_id", "movie_id"]]
assembler = VectorAssembler(inputCols=input_cols, outputCol=features_col)

print(input_cols)

['year_rated', 'gender_feature', 'occupation_feature', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', 'year_released', 'rated_released_year_diff', 'avg(unknown)', 'avg(Action)', 'avg(Adventure)', 'avg(Animation)', "avg(Children's)", 'avg(Comedy)', 'avg(Crime)', 'avg(Documentary)', 'avg(Drama)', 'avg(Fantasy)', 'avg(Film-Noir)', 'avg(Horror)', 'avg(Musical)', 'avg(Mystery)', 'avg(Romance)', 'avg(Sci-Fi)', 'avg(Thriller)', 'avg(War)', 'avg(Western)', 'popularity', 'avg(rating)']


In [48]:
data = assembler.transform(final_df)
data = data.select(answer_col,features_col, 'user_id', 'movie_id')
final_df.unpersist()
data.show(truncate=False)

+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+--------+
|rating|features                                                                                                                                                                                                                                                                                                                                                                                                                     |user_id|movie_id|
+------+----------------------------------------------------------------------------------------------------------------

In [49]:
data = data.dropna()
data.cache()

DataFrame[rating: int, features: vector, user_id: int, movie_id: int]

In [53]:
data.rdd.getNumPartitions()
data = data.repartition(1)
data.rdd.getNumPartitions()

1

In [54]:
data.count()

99991

In [55]:
training, test = data.randomSplit([0.8, 0.2])

In [56]:
rf = RandomForestRegressor(labelCol=answer_col)

In [57]:
model = rf.fit(training)

In [58]:
test_results = model.transform(test)

In [59]:
test_results.show()

+------+--------------------+-------+--------+------------------+
|rating|            features|user_id|movie_id|        prediction|
+------+--------------------+-------+--------+------------------+
|     1|(64,[0,1,2,23,24,...|    774|     398|1.5155091852055536|
|     1|(64,[0,1,2,23,24,...|    425|     398|2.4942595782980277|
|     1|(64,[0,1,2,23,24,...|    774|     231| 2.022574524378514|
|     1|(64,[0,1,2,23,24,...|    363|      29|2.6891749202913977|
|     1|(64,[0,1,2,23,24,...|    246|      29|2.5481146031584467|
|     1|(64,[0,1,2,23,24,...|    466|     231| 2.960839878748644|
|     1|(64,[0,1,2,23,24,...|    110|     173|3.7011180008527687|
|     1|(64,[0,1,2,23,24,...|    246|     802|2.5084469696318568|
|     1|(64,[0,1,2,23,24,...|    484|      24| 4.068495449616922|
|     1|(64,[0,1,2,23,24,...|    153|     172|3.4909181738509965|
|     1|(64,[0,1,2,23,24,...|    943|     720|3.0659151850728548|
|     1|(64,[0,1,2,23,24,...|    276|     769|2.6894745478617073|
|     1|(6

In [60]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import round

In [61]:
mae_eval = RegressionEvaluator(labelCol=answer_col, metricName='mae')
# mae_eval = MulticlassClassificationEvaluator(labelCol=answer_col)

In [62]:
mae = mae_eval.evaluate(test_results)

In [63]:
print(mae)

0.7492396741942997


In [64]:
rounded_test_results = test_results.withColumn('prediction', round(test_results.prediction))

In [65]:
rounded_test_results.show()

+------+--------------------+-------+--------+----------+
|rating|            features|user_id|movie_id|prediction|
+------+--------------------+-------+--------+----------+
|     1|(64,[0,1,2,23,24,...|    774|     398|       2.0|
|     1|(64,[0,1,2,23,24,...|    425|     398|       2.0|
|     1|(64,[0,1,2,23,24,...|    774|     231|       2.0|
|     1|(64,[0,1,2,23,24,...|    363|      29|       3.0|
|     1|(64,[0,1,2,23,24,...|    246|      29|       3.0|
|     1|(64,[0,1,2,23,24,...|    466|     231|       3.0|
|     1|(64,[0,1,2,23,24,...|    110|     173|       4.0|
|     1|(64,[0,1,2,23,24,...|    246|     802|       3.0|
|     1|(64,[0,1,2,23,24,...|    484|      24|       4.0|
|     1|(64,[0,1,2,23,24,...|    153|     172|       3.0|
|     1|(64,[0,1,2,23,24,...|    943|     720|       3.0|
|     1|(64,[0,1,2,23,24,...|    276|     769|       3.0|
|     1|(64,[0,1,2,23,24,...|    773|     769|       3.0|
|     1|(64,[0,1,2,23,24,...|    153|      50|       4.0|
|     1|(64,[0

In [66]:
accuracy_eval = MulticlassClassificationEvaluator(labelCol=answer_col)

In [67]:
accuracy = accuracy_eval.evaluate(rounded_test_results)

In [68]:
print(accuracy)

0.3157825115372181


In [69]:
from pyspark.sql.functions import floor

In [70]:
floor_test_results = test_results.withColumn('prediction', floor(test_results.prediction).cast("double"))

In [71]:
floor_test_results.show()

+------+--------------------+-------+--------+----------+
|rating|            features|user_id|movie_id|prediction|
+------+--------------------+-------+--------+----------+
|     1|(64,[0,1,2,23,24,...|    774|     398|       1.0|
|     1|(64,[0,1,2,23,24,...|    425|     398|       2.0|
|     1|(64,[0,1,2,23,24,...|    774|     231|       2.0|
|     1|(64,[0,1,2,23,24,...|    363|      29|       2.0|
|     1|(64,[0,1,2,23,24,...|    246|      29|       2.0|
|     1|(64,[0,1,2,23,24,...|    466|     231|       2.0|
|     1|(64,[0,1,2,23,24,...|    110|     173|       3.0|
|     1|(64,[0,1,2,23,24,...|    246|     802|       2.0|
|     1|(64,[0,1,2,23,24,...|    484|      24|       4.0|
|     1|(64,[0,1,2,23,24,...|    153|     172|       3.0|
|     1|(64,[0,1,2,23,24,...|    943|     720|       3.0|
|     1|(64,[0,1,2,23,24,...|    276|     769|       2.0|
|     1|(64,[0,1,2,23,24,...|    773|     769|       2.0|
|     1|(64,[0,1,2,23,24,...|    153|      50|       3.0|
|     1|(64,[0

In [72]:
accuracy = accuracy_eval.evaluate(floor_test_results)

In [73]:
print(accuracy)

0.25323695738311675


In [74]:
nb = NaiveBayes(labelCol=answer_col)
model = nb.fit(training)
test_results = model.transform(test)
mce = MulticlassClassificationEvaluator(labelCol=answer_col)
acc = mce.evaluate(test_results)
print(acc)

0.17823669394344605


In [75]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol=answer_col, numTrees=50, maxDepth=10, seed=42)
model = rf.fit(training)
test_results = model.transform(test)
mce = MulticlassClassificationEvaluator(labelCol=answer_col)
acc = mce.evaluate(test_results)
print(acc)

0.42029897060960936


In [85]:
from pyspark.mllib.evaluation import MulticlassMetrics
print(test_results.columns)
preds = test_results.select('rating', 'prediction').withColumn('rating_d', test_results['rating'].astype('double')).drop('rating')
print(preds.printSchema())
mm = MulticlassMetrics(preds.rdd)
confusion_matrix = mm.confusionMatrix()

['rating', 'features', 'user_id', 'movie_id', 'rawPrediction', 'probability', 'prediction']
root
 |-- prediction: double (nullable = false)
 |-- rating_d: double (nullable = true)

None


In [114]:
import numpy as np
con = np.array(confusion_matrix.values, dtype='int32').reshape((5,5))
print(con.sum())
print((con/con.sum()).round(5))
print((con/con.sum()).diagonal().sum())
con

19972
[[1.4720e-02 4.3600e-03 2.8000e-03 1.4500e-03 6.0000e-04]
 [3.5500e-03 4.7600e-03 3.0500e-03 7.5000e-04 2.0000e-04]
 [2.7140e-02 6.5990e-02 1.2142e-01 6.8000e-02 1.4470e-02]
 [1.4120e-02 3.6650e-02 1.3018e-01 2.3097e-01 1.2092e-01]
 [1.5000e-03 2.9000e-03 1.2870e-02 3.9560e-02 7.7060e-02]]
0.44892849989985983


array([[ 294,   87,   56,   29,   12],
       [  71,   95,   61,   15,    4],
       [ 542, 1318, 2425, 1358,  289],
       [ 282,  732, 2600, 4613, 2415],
       [  30,   58,  257,  790, 1539]])

In [94]:
confusion_matrix.asML()

DenseMatrix(5, 5, [294.0, 87.0, 56.0, 29.0, 12.0, 71.0, 95.0, 61.0, ..., 2600.0, 4613.0, 2415.0, 30.0, 58.0, 257.0, 790.0, 1539.0], 0)

In [101]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(labelCol=answer_col, numTrees=50, maxDepth=10, seed=42)
model = rf.fit(training)
test_results = model.transform(test)
re = RegressionEvaluator(metricName='rmse', labelCol=answer_col)
mae = re.evaluate(test_results)
print(mae)

0.0


In [116]:
print("\n".join(name + " - " + str(val) for name, val in zip(input_cols, model.featureImportances)))

year_rated - 0.0016229937602093506
gender_feature - 0.0012461027000109717
occupation_feature - 0.0013933791384578868
unknown - 0.0007862778511677879
Action - 0.0005193807346538636
Adventure - 0.0005373934030449095
Animation - 0.00038049018107633067
Children's - 0.00046318344202714605
Comedy - 0.00042866291873302814
Crime - 0.0006637737356761243
Documentary - 0.0004977402305733563
Drama - 0.0003242295005386404
Fantasy - 0.0005325043778420515
Film-Noir - 0.0003038022161542015
Horror - 0.0003149242812555039
Musical - 0.00033184660749104576
Mystery - 0.00036327631251334663
Romance - 0.00010994043280099081
Sci-Fi - 0.0002058549751313048
Thriller - 0.00013867138491034528
War - 0.00015676774455251305
Western - 9.412671081521299e-05
year_released - 0.0
rated_released_year_diff - 0.00596387256435291
avg(unknown) - 0.002644574686171624
avg(Action) - 0.0011788234557086373
avg(Adventure) - 0.002253492808967048
avg(Animation) - 0.004267906987181656
avg(Children's) - 0.0023477707235866475
avg(Comedy

In [105]:
re = RegressionEvaluator(metricName='rmse', labelCol=answer_col)
mae = re.evaluate(model.transform(training))
print("Train", mae, answer_col)

Train 0.8584752209725818 rating


In [106]:
re = RegressionEvaluator(metricName='mae', labelCol=answer_col)
mae = re.evaluate(test_results)
print(mae, answer_col)

0.7136277930335493 rating


In [115]:
re = RegressionEvaluator(metricName='rmse', labelCol=answer_col)
mae = re.evaluate(test_results)
print(mae, answer_col)

0.9005223543524787 rating


In [104]:
test_results.show(n=10)

+------+--------------------+-------+--------+------------------+
|rating|            features|user_id|movie_id|        prediction|
+------+--------------------+-------+--------+------------------+
|     1|(64,[0,1,2,23,24,...|    774|     398|1.2678262372263784|
|     1|(64,[0,1,2,23,24,...|    425|     398| 2.031977587407364|
|     1|(64,[0,1,2,23,24,...|    774|     231|1.5971554070974208|
|     1|(64,[0,1,2,23,24,...|    363|      29| 2.443847432533645|
|     1|(64,[0,1,2,23,24,...|    246|      29| 2.221701723523152|
|     1|(64,[0,1,2,23,24,...|    466|     231|2.6036329056100875|
|     1|(64,[0,1,2,23,24,...|    110|     173|3.8522454217675968|
|     1|(64,[0,1,2,23,24,...|    246|     802|2.2138041684316154|
|     1|(64,[0,1,2,23,24,...|    484|      24| 4.033611272755383|
|     1|(64,[0,1,2,23,24,...|    153|     172| 2.980772867102687|
+------+--------------------+-------+--------+------------------+
only showing top 10 rows



In [118]:
from pyspark.ml.regression import RandomForestRegressor
rf_full = RandomForestRegressor(labelCol=answer_col, numTrees=50, maxDepth=10, seed=42)
full_model = rf_full.fit(data)

In [131]:
avg_ratings_df = avg_ratings_df.repartition(1)
average_movie_rating = average_movie_rating.repartition(1)
popularity = popularity.repartition(1)
users = users.repartition(1)
movies = movies.repartition(1)

In [142]:
movies.describe().show()

+-------+------------------+--------------------+------------+------------------+--------------------+--------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+------------------+
|summary|          movie_id|         movie_title|release_date|video_release_date|            IMDb_URL|             unknown|             Action|         Adventure|          Animation|         Children's|             Comedy|              Crime|         Documentary|              Drama|             Fantasy|           Film-Noir|              Horror|            Musical|            Mystery|            Romance|             Sci-Fi|          Thriller|                War|            Western|     year_

In [146]:
def recommend(user_id, model, time=2018):
    from pyspark.sql.functions import lit
    d = users.filter(users.user_id == user_id).select('user_id','age','gender_feature','occupation_feature')
    print(d.columns)
    d = d.crossJoin(movies.drop('movie_title', 'release_date', 'video_release_date', 'IMDb_URL'))
    print(d.columns)
    #.show()
    d = d.repartition(1)
    d = d.withColumn('year_rated', lit(time))
    d = d.withColumn('rated_released_year_diff', d.year_rated - d.year_released)
    print(d.columns)
    d = d.fillna(1997)
    d.describe().show()
    #avg_ratings_df_p = avg_ratings_df.repartition(1)
    #average_movie_rating_p = average_movie_rating.repartition(1)
    #popularity_p = popularity.repartition(1)
    
    d = d.join(avg_ratings_df, 'user_id')
    d = d.join(average_movie_rating, 'movie_id')
    d = d.join(popularity, 'movie_id')
    d = d.dropna()
    assembler = VectorAssembler(inputCols=input_cols, outputCol=features_col)
    d = assembler.transform(d)
    d = d.select(features_col, 'user_id', 'movie_id')
    d = d.repartition(1)
    #print(d.count(), d.printSchema())
    
    d.show()
    predictions = model.transform(d)
    
    return predictions
preds = recommend(122, full_model)
preds.show()

['user_id', 'age', 'gender_feature', 'occupation_feature']
['user_id', 'age', 'gender_feature', 'occupation_feature', 'movie_id', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', 'year_released']
['user_id', 'age', 'gender_feature', 'occupation_feature', 'movie_id', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', 'year_released', 'year_rated', 'rated_released_year_diff']
+-------+-------+----+------------------+--------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+----------------

In [ ]:
from pyspark.ml.recommendation import ALS

In [ ]:
als = ALS(maxIter=5,regParam=0.01,userCol='user_id',itemCol='movie_id',ratingCol='rating')

In [ ]:
model = als.fit(training)

In [ ]:
test_results = model.transform(test)

In [ ]:
test_results.show()

In [ ]:
test_results = test_results.dropna(subset=['prediction'])

In [ ]:
mae = mae_eval.evaluate(test_results)

In [ ]:
print(mae)

In [ ]:
rounded_test_results = test_results.withColumn('prediction', round(test_results.prediction).cast('double'))

In [ ]:
rounded_test_results.show()

In [ ]:
acc = accuracy_eval.evaluate(rounded_test_results)

In [ ]:
print(acc)